In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import re
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import *
from torchvision import transforms
from torchvision.models import *

import random

def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Call the function to set the seeds
set_seeds()

In [3]:
class CustomDataset(Dataset):
    def __init__(self, image_dir, labels_file, transform=None, filenames=None, labels=None):
        self.image_dir = image_dir
        self.labels_df = pd.read_csv(labels_file)
        self.transform = transform

        # Use provided filenames and labels if available
        if filenames is not None and labels is not None:
            self.filenames = filenames
            self.labels = labels
            self.use_additional_data = True
        else:
            self.filenames = self.labels_df['id'].astype(str).values
            self.labels = self.labels_df['malignant'].values + 1
            self.use_additional_data = False

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if self.use_additional_data:
            img_path = os.path.join(self.image_dir, 'img_' + str(self.filenames[idx]) + '.png')
        else:
            img_path = os.path.join(self.image_dir, 'img_' + str(self.labels_df.iloc[idx]['id']) + '.png')

        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Set the directory paths
image_dir = '/kaggle/input/oxml-carinoma-classification'
labels_file = '/kaggle/input/oxml-carinoma-classification/labels.csv'


# Load the labeled dataset
labeled_data = pd.read_csv(labels_file)

# Get the labeled image filenames and their corresponding labels
labeled_filenames = labeled_data['id'].astype(str).values
labels = labeled_data['malignant'].values + 1

# Find the maximum size
max_width = 0
max_height = 0

for file_name in labeled_filenames:
    file_path = os.path.join(image_dir, "img_" + file_name + '.png')
    image = Image.open(file_path)
    width, height = image.size
    max_width = max(max_width, width)
    max_height = max(max_height, height)


# Define the main data transform
main_transform = transforms.Compose([
    transforms.Resize((max_height, max_width)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create the main dataset
dataset = CustomDataset(image_dir, labels_file, transform=main_transform)

# Define data augmentation transforms
augmentation_transform = transforms.Compose([
    transforms.Resize((max_height, max_width)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(45),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset_augmented = CustomDataset(image_dir, labels_file, transform=augmentation_transform)

# Define data augmentation transforms 2
augmentation_transform_2 = transforms.Compose([
    transforms.Resize((max_height, max_width)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.RandomResizedCrop((max_height, max_width), scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomPerspective(distortion_scale=0.1, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset_augmented_2 = CustomDataset(image_dir, labels_file, transform=augmentation_transform_2)


stacked_dataset = ConcatDataset([dataset, dataset_augmented])
stacked_labels = np.concatenate([labels, labels])


# Define the number of folds for k-fold cross-validation
k_folds = 8

# Perform k-fold cross-validation
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
fold_scores = []

for fold, (train_index, val_index) in enumerate(skf.split(stacked_dataset, stacked_labels)):
    print(f"Training on fold {fold + 1}/{k_folds}")

    # Split the dataset into train and validation sets for the current fold
    train_data = [stacked_dataset[idx] for idx in train_index]
    val_data = [stacked_dataset[idx] for idx in val_index]


    # Convert labels to numpy array for indexing
    stacked_labels = np.array(stacked_labels)

    # Calculate class weights
    class_weights = 1.0 / torch.tensor(np.bincount(stacked_labels[train_index]))
    train_class_weights = class_weights[stacked_labels[train_index]]

    # Create the weighted sampler
    sampler = WeightedRandomSampler(train_class_weights, len(train_data), replacement=True)

    # Create the data loaders with weighted sampling
    batch_size = 8
    train_dataloader = DataLoader(train_data, batch_size=batch_size, sampler=sampler)
    val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

Training on fold 1/8
Training on fold 2/8
Training on fold 3/8
Training on fold 4/8
Training on fold 5/8
Training on fold 6/8
Training on fold 7/8
Training on fold 8/8


In [4]:
from torchvision import models

# Load pre-trained models
resnet_model = models.resnet50(pretrained=True)
efficientnet_model = models.efficientnet_v2_s(pretrained=True)
inception_model = models.inception_v3(pretrained=True, aux_logits=True)
googlenet_model = models.googlenet(pretrained=True)

# Freeze all the parameters of the pretrained models
for param in resnet_model.parameters():
    param.requires_grad = False
for param in efficientnet_model.parameters():
    param.requires_grad = False
for param in inception_model.parameters():
    param.requires_grad = False
for param in googlenet_model.parameters():
    param.requires_grad = False

# Replace the last fully connected layers to match the number of classes
num_classes = 3
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, num_classes)
efficientnet_model.classifier[-1] = nn.Linear(efficientnet_model.classifier[-1].in_features, num_classes)
inception_model.fc = nn.Linear(inception_model.fc.in_features, num_classes)
googlenet_model.fc = nn.Linear(googlenet_model.fc.in_features, num_classes)

# Move the models to the device (GPU if available)
resnet_model = resnet_model.to(device)
efficientnet_model = efficientnet_model.to(device)
inception_model = inception_model.to(device)
googlenet_model = googlenet_model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizers for each model
optimizer_resnet = optim.Adam(resnet_model.parameters(), lr=0.001)
optimizer_efficientnet = optim.Adam(efficientnet_model.parameters(), lr=0.001)
optimizer_inception = optim.Adam(inception_model.parameters(), lr=0.001)
optimizer_googlenet = optim.Adam(googlenet_model.parameters(), lr=0.001)

# Set the initial best validation loss and accuracy
best_val_loss = float('inf')
best_val_f1 = 0.0
best_val_acc = 0.0

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    # Training phase for ResNet50
    resnet_model.train()
    running_loss = 0.0
    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer_resnet.zero_grad()
        outputs = resnet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_resnet.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - ResNet50 Training Loss: {epoch_loss:.4f}")

    # Training phase for EfficientNetV2
    efficientnet_model.train()
    running_loss = 0.0
    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer_efficientnet.zero_grad()
        outputs = efficientnet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_efficientnet.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - EfficientNetV2 Training Loss: {epoch_loss:.4f}")

    # Training phase for Inception V3
    inception_model.train()
    running_loss = 0.0
    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = inception_model(images)
        logits = outputs.logits  # Get the output logits

        loss = criterion(logits, labels)

        # Backward pass and optimization
        optimizer_inception.zero_grad()
        loss.backward()
        optimizer_inception.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Inception V3 Training Loss: {epoch_loss:.4f}")

    # Training phase for GoogLeNet
    googlenet_model.train()
    running_loss = 0.0
    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer_googlenet.zero_grad()
        outputs = googlenet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_googlenet.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - GoogLeNet Training Loss: {epoch_loss:.4f}")

    # Validation phase
    resnet_model.eval()
    efficientnet_model.eval()
    inception_model.eval()
    googlenet_model.eval()

    val_loss = 0.0
    val_f1 = 0.0
    val_accuracy = 0.0
    total_predictions = torch.tensor([], dtype=torch.long, device=device)  # Initialize total_predictions
    total_labels = torch.tensor([], dtype=torch.long, device=device)  # Initialize total_labels

    with torch.no_grad():
        for images, labels in val_dataloader:
            images = images.to(device)
            labels = labels.to(device)

            # ResNet50
            resnet_outputs = resnet_model(images)
            resnet_loss = criterion(resnet_outputs, labels)

            # EfficientNetV2
            efficientnet_outputs = efficientnet_model(images)
            efficientnet_loss = criterion(efficientnet_outputs, labels)

            # Inception V3
            inception_outputs = inception_model(images)
            inception_loss = criterion(inception_outputs, labels)

            # GoogLeNet
            googlenet_outputs = googlenet_model(images)
            googlenet_loss = criterion(googlenet_outputs, labels)

            # Combine losses
            loss = (resnet_loss + efficientnet_loss + inception_loss + googlenet_loss) / 4.0

            val_loss += loss.item()
            
            # Calculate accuracy
            _, predictions = torch.max((resnet_outputs + efficientnet_outputs + inception_outputs + googlenet_outputs) / 4.0, 1)
            val_accuracy += torch.sum(predictions == labels).item()

            # Calculate F1-score
            predictions = (resnet_outputs + efficientnet_outputs + inception_outputs + googlenet_outputs) / 4.0
            predictions = torch.argmax(predictions, dim=1)

            total_predictions = torch.cat((total_predictions, predictions), dim=0)
            total_labels = torch.cat((total_labels, labels), dim=0)

    val_loss /= len(val_dataloader)
    val_f1 = f1_score(total_labels.cpu(), total_predictions.cpu(), average='weighted')  # Calculate F1-score outside the loop
    val_accuracy /= len(val_dataloader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs} - Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, F1 Score: {val_f1:.4f}")
    # Checkpointing based on F1 score
    if val_accuracy > best_val_acc :
        best_val_acc  = val_accuracy
        print(f"Best Ever Accuracy: {best_val_acc}")
        torch.save(resnet_model.state_dict(), 'best_resnet_model.pth')
        torch.save(efficientnet_model.state_dict(), 'best_efficientnet_model.pth')
        torch.save(inception_model.state_dict(), 'best_inception_model.pth')
        torch.save(googlenet_model.state_dict(), 'best_googlenet_model.pth')

print("Training complete!")

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 229MB/s]
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed i

Epoch 1/3 - ResNet50 Training Loss: 1.1410
Epoch 1/3 - EfficientNetV2 Training Loss: 1.0820
Epoch 1/3 - Inception V3 Training Loss: 1.1715
Epoch 1/3 - GoogLeNet Training Loss: 1.1122
Epoch 1/3 - Validation Loss: 1.1064, Accuracy: 0.2667, F1 Score: 0.1906
Best Ever Accuracy: 0.26666666666666666
Epoch 2/3 - ResNet50 Training Loss: 1.0625
Epoch 2/3 - EfficientNetV2 Training Loss: 1.0837
Epoch 2/3 - Inception V3 Training Loss: 1.1341
Epoch 2/3 - GoogLeNet Training Loss: 1.0987
Epoch 2/3 - Validation Loss: 1.0511, Accuracy: 0.7333, F1 Score: 0.7133
Best Ever Accuracy: 0.7333333333333333
Epoch 3/3 - ResNet50 Training Loss: 0.9298
Epoch 3/3 - EfficientNetV2 Training Loss: 1.0511
Epoch 3/3 - Inception V3 Training Loss: 1.1163
Epoch 3/3 - GoogLeNet Training Loss: 1.0534
Epoch 3/3 - Validation Loss: 1.0391, Accuracy: 0.6000, F1 Score: 0.5371
Training complete!


In [5]:
import os
import pandas as pd
import torch
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torchvision.models as models
from torchvision.transforms import transforms

class CustomDataset(Dataset):
    def __init__(self, image_dir, labeled_images, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.filenames = [file for file in os.listdir(image_dir) if file.endswith('.png') and file not in labeled_images]

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        file_name = self.filenames[idx]
        img_path = os.path.join(self.image_dir, file_name)

        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, file_name

# Set random seed for reproducibility
torch.manual_seed(42)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the directory paths
image_dir = '/kaggle/working/padded_images'
image_dir = '/kaggle/input/oxml-carinoma-classification'

# Load labeled images from labels.csv
labeled_images = labeled_data['id'].astype(str).values
labeled_images ='img_' + labeled_images + '.png'
# Define the main data transform
main_transform = transforms.Compose([
    transforms.Resize((max_height, max_width)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create the main dataset
dataset = CustomDataset(image_dir, labeled_images, transform=main_transform)

# Create the data loader
batch_size = 8
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Load the trained model weights
resnet_model.load_state_dict(torch.load('best_resnet_model.pth'))
efficientnet_model.load_state_dict(torch.load('best_efficientnet_model.pth'))
inception_model.load_state_dict(torch.load('best_inception_model.pth'))
googlenet_model.load_state_dict(torch.load('best_googlenet_model.pth'))

# Testing phase
resnet_model.eval()
efficientnet_model.eval()
inception_model.eval()
googlenet_model.eval()

predictions = []

with torch.no_grad():
    for images, filenames in dataloader:
        images = images.to(device)

        # ResNet50
        resnet_outputs = resnet_model(images)

        # EfficientNetV2
        efficientnet_outputs = efficientnet_model(images)

        # Inception V3
        inception_outputs = inception_model(images)

        # GoogLeNet
        googlenet_outputs = googlenet_model(images)

        # Combine predictions
        combined_outputs = (resnet_outputs + efficientnet_outputs + inception_outputs + googlenet_outputs) / 4.0

        predicted_labels = torch.argmax(combined_outputs, dim=1)
        predicted_labels = predicted_labels.cpu().detach().numpy() - 1

        for filename, label in zip(filenames, predicted_labels):
            predictions.append([filename, label])

# Create a DataFrame for the predictions
output = pd.DataFrame(predictions, columns=['id', 'malignant'])
# Convert the 'id' column to integer type
output['id'] = output['id'].str.replace('.png', '', regex=False).str.replace('img_', '', regex=False).astype(int)

# Sort the output DataFrame by "id"
output.sort_values(by="id", inplace=True)

# Save the predictions to a CSV file
output.to_csv('/kaggle/working/submission.csv', index=False)

print("Predictions saved successfully!")


Predictions saved successfully!


In [6]:
pd.read_csv('submission.csv')


,id,malignant
0,5042,-1
1,28148,-1
2,35946,1
3,37944,0
4,39157,-1
...,...,...
119,959476,-1
120,968389,-1
121,976505,1
122,996288,-1
